In [ ]:
!pip install opencv-contrib-python==3.4.0.12 
# otherwise you'd get an error message saying this when you try to use SIFT
# This algorithm is patented and is excluded in this configuration; 
# Set OPENCV_ENABLE_NONFREE CMake option and rebuild the library in 
# function 'cv::xfeatures2d::SIFT::create'

In [ ]:
import numpy as np
import pandas as pd

from scipy import ndimage, signal
from scipy.ndimage import gaussian_filter
from copy import deepcopy
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx

import random
import math
import os

from copy import deepcopy

!pip install opencv-python
import cv2

import plotly.graph_objects as go

from google.colab.patches import cv2_imshow

In [ ]:
def get_images(path0, path1):
  img0 = cv2.imread(path0, cv2.IMREAD_GRAYSCALE)
  img1 = cv2.imread(path1, cv2.IMREAD_GRAYSCALE)

  return img0, img1

In [ ]:
def get_point_matches(img1, img2, metric='SIFT', threshold=0.5):
  if metric == 'SIFT':
    descriptor = cv2.xfeatures2d.SIFT_create()
  elif metric == 'SURF':
    descriptor = cv2.xfeatures2d.SURF_create()
  else:
    descriptor = cv2.ORB_create(nfeatures=1000)
  
  kp1, desc1 = descriptor.detectAndCompute(img1, None)
  kp2, desc2 = descriptor.detectAndCompute(img2, None)

  bf = cv2.BFMatcher()
  matches = bf.knnMatch(desc1, desc2, k=2) # k=2 means find the top two matchs for each query descriptor

  # Apply ratio test (as per David Lowe's SIFT paper: compare the best match with the 2nd best match_
  good_matches = []
  good_matches_without_list = []
  for m,n in matches:
      if m.distance < threshold*n.distance: # only accept matchs that are considerably better than the 2nd best match
          good_matches.append([m])
          good_matches_without_list.append(m) # this is to simplify finding a homography later


  src_pts = np.float32([ kp1[m.queryIdx].pt for m in good_matches_without_list ]).reshape(-1,1,2)
  dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good_matches_without_list ]).reshape(-1,1,2)

  src_dst_pts = np.concatenate((src_pts.squeeze(), dst_pts.squeeze()), axis=1)
  src_dst_pts = src_dst_pts[np.argsort(src_dst_pts[:,0])]

  src_pts = src_dst_pts[:,0:2]
  dst_pts = src_dst_pts[:,2:]

  return src_pts, dst_pts

In [ ]:
def extract_calibration_vals(file, image_numbers):
    image1, image2 = image_numbers

    calib_vals = {}
    temple_par = pd.read_csv("dino_par.txt", delimiter=' ', dtype=str, skiprows=[0], index_col=0, header=None)
    temple_par.columns = ['k11', 'k12', 'k13', 'k21', 'k22', 'k23', 
                          'k31', 'k32', 'k33', 'r11', 'r12', 'r13', 
                          'r21', 'r22', 'r23', 'r31', 'r32', 'r33', 
                          't1', 't2', 't3']
    temple_par.index.name = 'image name'
    K0 = temple_par.iloc[image1,0:9].values.reshape(3,3).astype(float)
    R0 = temple_par.iloc[image1,9:18].values.reshape(3,3).astype(float)
    t0 = temple_par.iloc[image1,18:].values.reshape(3,1).astype(float)
    K1 = temple_par.iloc[image2,0:9].values.reshape(3,3).astype(float)
    R1 = temple_par.iloc[image2,9:18].values.reshape(3,3).astype(float)
    t1 = temple_par.iloc[image2,18:].values.reshape(3,1).astype(float)
    calib_vals['cam0'] = K0
    calib_vals['cam1'] = K1
    calib_vals['R0'] = R0
    calib_vals['R1'] = R1
    calib_vals['t0'] = t0
    calib_vals['t1'] = t1
    calib_vals['baseline'] = np.linalg.norm(t1 - t0)
    return calib_vals

In [ ]:
def drawlines(img1, img2, lines, pts1, pts2, colours):
    ''' img1 - image on which we draw the epilines for the points in img2
        lines - corresponding epilines '''
    r,c = img1.shape
    img1 = img1.transpose()#[::-1,:]
    img2 = img2.transpose()#[::-1,:]
    img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
    for index, (r, pt1, pt2) in enumerate(zip(lines, pts1, pts2)):
        colour = colours[index]
        x0,y0 = map(int, [0, -r[2]/r[1] ])
        x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
        img1 = cv2.line(img1, (y0,x0), (y1,x1), colour, 1)
        img1 = cv2.circle(img1, tuple(pt1)[::-1], 5, colour, -1)
        img2 = cv2.circle(img2, tuple(pt2)[::-1], 5, colour, -1)
    return img1, img2

In [ ]:
def compute_epipolar_lines(F, img1, img2, pts1, pts2, image_path_0, image_path_1):
  # Find epilines corresponding to points in right image (second image) and
  # drawing its lines on left image

  # Acquire colors:
  rainbow_cm = plt.get_cmap('rainbow')
  c_norm  = colors.Normalize(vmin=0, vmax=len(pts1)-1)
  scalar_map = cmx.ScalarMappable(norm=c_norm, cmap=rainbow_cm)
  
  colours = []
  for i in range(len(pts1)):
    colours.append(np.asarray(scalar_map.to_rgba(i)[:3])*255)

  lines1 = cv2.computeCorrespondEpilines(pts2.reshape(-1,1,2), 2,F)
  lines1 = lines1.reshape(-1,3)
  img5,img6 = drawlines(img1, img2, lines1, pts1, pts2, colours)
  # Find epilines corresponding to points in left image (first image) and
  # drawing its lines on right image
  lines2 = cv2.computeCorrespondEpilines(pts1.reshape(-1,1,2), 1,F)
  lines2 = lines2.reshape(-1,3)
  img3,img4 = drawlines(img2, img1, lines2, pts2, pts1, colours)

  fig, axs = plt.subplots(1, 2, figsize=(20, 10))
  axs[0].imshow(img5)
  axs[0].set_ylim(0, img1.shape[1])
  axs[0].set_title('Epipolar Lines of image {}'.format(image_path_0))
  axs[1].imshow(img3)
  axs[1].set_ylim(0, img1.shape[1])
  axs[1].set_title('Epipolar Lines of image {}'.format(image_path_1))
  
  plt.show()

In [ ]:
def open_cv_homography(img0, img1, F, img0_pts, img1_pts, suppress_images):
  thresh = 0
  height, width = img0.shape
  _, H0, H1 = cv2.stereoRectifyUncalibrated(
      np.float32(img0_pts), np.float32(img1_pts), F, imgSize=(width, height), threshold=thresh,
  )

  img0_rectified = cv2.warpPerspective(img0, H0, (width, height))
  img1_rectified = cv2.warpPerspective(img1, H1, (width, height))
  if not suppress_images:
    cv2_imshow(img0_rectified)
    cv2_imshow(img1_rectified)

  return H0, H1, img0_rectified, img1_rectified

In [ ]:
def normalize_coordinates(img_pts):
    n = len(img_pts)
    # compute centroid in image
    centroid_x, centroid_y = img_pts.mean(0)
    O = np.array([[1, 0, -centroid_x],
                  [0, 1, -centroid_y],
                  [0, 0, 1]])
    
    # recenter
    recentered_pts = img_pts - [centroid_x,centroid_y]
    
    # find scale term
    s = np.sqrt(2) / np.sqrt(1/n * np.sum(recentered_pts**2))
    S = np.array([[s, 0, 0],
                  [0, s, 0],
                  [0, 0, 1]])

    # transformation matrix
    T = S @ O
    
    # new coordinates
    norm_pts = np.empty(img_pts.shape)
    for idx, pt in enumerate(img_pts):
        norm_pt = T @ np.array([[pt[0]], [pt[1]], [1]])
        norm_pt = (norm_pt[:2]).squeeze()
        norm_pts[idx] = norm_pt
    return T, norm_pts #return T s.t. we can convert back later

In [ ]:
def eight_point(src_pts, dst_pts, T0, T1):
    # 8-point algorithm
    num_match = src_pts.shape[0]
    A = np.empty((num_match, 9))
    for idx in range(num_match):
        x0, y0 = src_pts[idx]
        x1, y1 = dst_pts[idx]
        A[idx] = [x1*x0, x1*y0, x1, y1*x0, y1*y0, y1, x0, y0, 1]

    # use SVD to solve F -- take column of V corresponding to least singular value
    U,S,VT = np.linalg.svd(A)
    V = VT.T
    F = V[:,-1].reshape(3,3)

    # enforce rank 2 constraint (Tsai and Huang) -- make last eigval 0
    U,S,VT = np.linalg.svd(F)
    S[-1] = 0
    F = U@(np.diag(S))@VT

    # need to un-normalize F
    F = T1.T @ F @ T0
    return F/F[2,2]

In [ ]:
def get_fundamental_matrix(parameters, F_method):
  f, K0, K1, R0, R1, t0, t1, img0_pts, img1_pts = parameters

  # fundamental matrix
  if F_method == 'Hartley':
    R1_camera_coord_0 = R1 @ R0.transpose()
    t1_camera_coord_0 = t0 - R1_camera_coord_0.transpose() @ t1

    KRt = K0 @ R1_camera_coord_0 @ t1_camera_coord_0
    cross_product_matrix_KRt = np.asarray([[0, -KRt[2][0], KRt[1][0]], [KRt[2][0], 0, -KRt[0][0]], [-KRt[1][0], KRt[0][0], 0]])
    F = cross_product_matrix_KRt @ (K1 @ R1) @ np.linalg.inv(K0 @ R0)

  elif F_method == 'openCV':
    F, _ = cv2.findFundamentalMat(img0_pts, img1_pts, cv2.FM_RANSAC)

  else:
    T0, img0_pts_norm = normalize_coordinates(img0_pts)
    T1, img1_pts_norm = normalize_coordinates(img1_pts)
    F = eight_point(img0_pts_norm, img1_pts_norm, T0, T1)

  return F

In [ ]:
def rectify_image(img0, img1, parameters, image_path_0, image_path_1, F_method='Hartley', suppress_images=True):
  f, K0, K1, R0, R1, t0, t1, img0_pts, img1_pts = parameters
  
  F = get_fundamental_matrix(parameters, F_method=F_method)
  #print(F/F[-1,-1])

  if not suppress_images:
    compute_epipolar_lines(F, img0, img1, img0_pts, img1_pts, image_path_0, image_path_1)

  H0, H1, img0_rectified, img1_rectified = open_cv_homography(img0, img1, F, img0_pts, img1_pts, suppress_images)

  print("Sanity Check")

  Fbar = np.asarray([[0,0,0],[0,0,-1],[0,1,0]])
  F_recap = H1.transpose() @ Fbar @ H0
  print("Ensure the following matrices are equal [H1.T @ Fbar @ H0 = F]")
  print(F_recap / F_recap[-1,-1])
  print(F/F[-1,-1])

  return (H0, H1, img0_rectified, img1_rectified)

In [ ]:
# remove points that are n stdev away from the mean
def remove_outliers(world_coord, num_std=3):
  mean_coord = np.array([np.mean(world_coord[:,0]), np.mean(world_coord[:,1]), np.mean(world_coord[:,2])])
  stdev_coord = np.array([np.std(world_coord[:,0]), np.std(world_coord[:,1]), np.std(world_coord[:,2])])

  world_coord = world_coord[(world_coord[:,0] > mean_coord[0] - num_std*stdev_coord[0]) & (world_coord[:,0] < mean_coord[0] + num_std*stdev_coord[0])]
  world_coord = world_coord[(world_coord[:,1] > mean_coord[1] - num_std*stdev_coord[1]) & (world_coord[:,1] < mean_coord[1] + num_std*stdev_coord[1])]
  world_coord = world_coord[(world_coord[:,2] > mean_coord[2] - num_std*stdev_coord[2]) & (world_coord[:,2] < mean_coord[2] + num_std*stdev_coord[2])]

  return world_coord

In [ ]:
# Get the 3D coordinates of the matches
def get_world_coord(img0, img1, rgb0, parameters, match_method='SIFT'):
  f, K0, K1, R0, R1, t0, t1, img0_pts, img1_pts = parameters

  px0 = K0[0,2]
  px1 = K1[0,2]
  py0 = K0[1,2]
  py1 = K1[1,2]
  
  T = np.linalg.norm([px0-px1, py0-py1])  #calib_vals.get('doffs', np.linalg.norm([px0-px1, py0-py1]))

  img0_pts = np.array(img0_pts)
  img1_pts = np.array(img1_pts)

  factor = 1

  # Calculate world coordinates per pixel
  num_pts = len(img0_pts)
  world_coord = np.zeros((num_pts,6))
  for idx, pt in enumerate(img0_pts):
    # Calculate depth using: z = fT/(xr-xl)
    z = f*T/(np.absolute((pt[0])-(img1_pts[idx,0]))*factor)
    world_coord[idx,2] = z

    # x = fX/z + px, solve for X where X is world coordinate of x in the img
    # X = (x-px)z/f
    x0 = (pt[0]-px0)*factor*z/f
    x1 = (img1_pts[idx,0]-px1)*factor*z/f
    world_coord[idx,0] = -x0
    #world_coord[idx,0] = -(x0+x1)/2
    # do same for Y
    y0 = (pt[1]-py0)*factor*z/f
    y1 = (img1_pts[idx,1]-py1)*factor*z/f
    world_coord[idx,1] = y0
    #world_coord[idx,1] = (y0+y1)/2

    # add colour
    world_coord[idx,3:] = rgb0[int(pt[1])*factor,int(pt[0])*factor,:]

  return world_coord

In [ ]:
# plot 3d point cloud of matches via plotly
def plot_pointCloud(pc, path='plot.html'):
    '''
    plots the Nx6 point cloud pc in 3D
    assumes (1,0,0), (0,1,0), (0,0,-1) as basis
    '''
    fig = go.Figure(data=[go.Scatter3d(
        x=pc[:, 0],
        y=pc[:, 1],
        z=-pc[:, 2],
        mode='markers',
        marker=dict(
            size=2,
            color=pc[:, 3:][..., ::-1],
            opacity=0.8
        )
    )])
    fig.write_html(path)
    fig.show()


In [69]:
def run(image_index_0, image_index_1, calibration_filepath, match_method='SIFT', threshold=0.75, outlier_bound=2, do_rectify=True, F_method='Hartley', suppress_images=True):
  base_image_path = "dino{:04d}.png"

  image_path_0 = base_image_path.format(image_index_0)
  image_path_1 = base_image_path.format(image_index_1)

  img0, img1 = get_images(image_path_0, image_path_1)
  rgb0 = cv2.imread(image_path_0)

  # Acquire camera information
  calib_vals = extract_calibration_vals(calibration_filepath, (image_index_0, image_index_1))
  K0 = calib_vals['cam0']
  K1 = calib_vals['cam1']
  f = K0[0,0]
  
  R0 = calib_vals['R0']
  R1 = calib_vals['R1']

  t0 = calib_vals['t0']
  t1 = calib_vals['t1']

  # Extract matching points from the imgs
  img0_pts, img1_pts = get_point_matches(img0, img1, metric=match_method, threshold=threshold)

  parameters = (f, K0, K1, R0, R1, t0, t1, img0_pts.squeeze(), img1_pts.squeeze())

  if do_rectify:
    rectify_output = rectify_image(img0, img1, parameters, image_path_0, image_path_1, F_method=F_method, suppress_images=suppress_images)

    H0, H1 = rectify_output[:2]
    img0 = rectify_output[2]
    img1 = rectify_output[3]

    #K0 = H0 @ K0
    #K1 = H1 @ K1
    #R0 = R0 @ H0
    #R1 = R1 @ H1
    #t0 = H0 @ t0
    #t1 = H1 @ t1

    #f = K0[0,0]

    #print(R0/R0[-1,-1])
    #print(R1/R1[-1,-1])

    #img0_pts, img1_pts = get_point_matches(img0, img1, metric=match_method, threshold=threshold)

    #parameters = (f, K0, K1, R0, R1, t0, t1, img0_pts.squeeze(), img1_pts.squeeze())

  world_coord = get_world_coord(img0, img1, rgb0, parameters, match_method)
  #world_coord = remove_outliers(world_coord, outlier_bound)
  plot_pointCloud(world_coord)



In [71]:
run(image_index_0=2, image_index_1=1, calibration_filepath='dino_par.txt', match_method='SURF', threshold=0.75, outlier_bound=2, do_rectify=False, F_method='Hartley', suppress_images=True)